<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/spaCy_NER_Training_(transformer).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U spacy-transformers
!python -m spacy download zh_core_web_trf
!pip install spacy-pkuseg

     |████████████████████████████████| 2.2 MB 15.8 MB/s 
     |████████████████████████████████| 6.4 MB 45.3 MB/s 
     |████████████████████████████████| 998 kB 57.4 MB/s 
     |████████████████████████████████| 456 kB 47.6 MB/s 
     |████████████████████████████████| 10.1 MB 44.2 MB/s 
     |████████████████████████████████| 621 kB 45.6 MB/s 
     |████████████████████████████████| 42 kB 1.8 MB/s 
     |████████████████████████████████| 895 kB 36.7 MB/s 
     |████████████████████████████████| 3.3 MB 25.9 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninst

In [11]:
sentences = [
  '恆指上升3.3%，是香港在這半年最大漲幅；恒生科技指數漲7.9%，創下去年7月高位。',
  '中電在假期前做好帶動，恒指單日暴升961.22點。',
  '受美股收跌拖累，匯控、和記跌超過2%。',
  '匯控下跌2%。',
  '日本、新加坡、澳洲股市單日暴升961.22點。'
]

TRAIN_DATA = [
  (
    '現時恒指已跌穿重要支持位26500點來看，港股繼續尋底的機會是頗高的。',
    {
        'entities':
        [
         (2,4,'FIN'),(9,12,'FIN'),(21,23,'FIN'),(25,27,'FIN')
        ]
    }
  ),
  (
    '恆生指數和創業板指數。',
    {
      'entities': 
      [
        (0,4,'FIN'),(5,10,'FIN')          
      ]
    }
  ),  
  (
    '中電、匯控、和黃8月暴升8.5%。',
    {
      'entities': 
      [
        (0,2,'STOCK'),(3,5,'STOCK'),(6,8,'STOCK'),(10,12,'FIN')         
      ]
    }
  ),
  (
    '恆指小雙底9月盼反彈。中電、匯控、和黃8月齊齊跌8.5%金融海嘯後最差。',
    {
      'entities': 
      [
        (0,2,'FIN'),(2,5,'FIN'),(8,10,'FIN'),(11,13,'STOCK'),(14,16,'STOCK'),(17,19,'STOCK')      
      ]
    }
  ),     
  (
    '匯控(00005)將於下周一（2日）公布2021年度中期業績。',
    {
        'entities':
        [
          (0,2,'STOCK'),(26,30,'FIN')
        ]
    }
  ),
  (
    '查看香港最新的股市報價、記錄、新聞及其他重要資訊，助你賣買股票及投資。',
   {
      'entities':
      [
        (2,4,'GPE'),(7,9,'FIN'),(9,11,'FIN')
      ]
   }
  ),
  (
    '匯控低開4%。港股收市跌。',
    {
        'entities':
        [
         (0,2,'STOCK'),(7,9,'FIN')
        ]
    }
  ),
  (
    '港股高開173點。',
   {
       'entities': [(0,2,'FIN'),(2,4,'FIN')]
   }
  )
]

In [12]:
import spacy
from spacy import displacy

nlp = spacy.load('zh_core_web_trf')

for s in sentences:
  doc = nlp(s)
  displacy.render(doc,jupyter=True,style="ent")

  # for ent in doc.ents:
  #   print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [13]:
import random
from spacy.training.example import Example
from spacy.util import minibatch, compounding

ner = nlp.get_pipe('ner')
ner.add_label('STOCK')
ner.add_label('FIN')

# Resume training
# https://www.machinelearningplus.com/nlp/training-custom-ner-model-in-spacy/
# optimizer = nlp.resume_training()
# move_names = list(ner.move_names)

optimizer=nlp.resume_training()
epoch = 20
batch_size = compounding(4.0, 32.0, 1.001)
for itn in range(epoch):
  random.shuffle(TRAIN_DATA)
  losses = {}
  # batch up the examples using spaCy's minibatch
  batches = spacy.util.minibatch(TRAIN_DATA, size=batch_size)
  for batch in batches:
    texts, annotations = zip(*batch)
    example = []
    for i in range(len(texts)):
      doc = nlp.make_doc(texts[i])
      example.append(Example.from_dict(doc, annotations[i]))
    nlp.update(example,drop=0.3,sgd=optimizer,losses=losses)
  # for text,annotations in TRAIN_DATA:
  #   doc = nlp.make_doc(text)
  #   example = Example.from_dict(doc,annotations)
  #   nlp.update([example],sgd=optimizer,losses=losses)
  
  print("Losses", losses)

# nlp.to_disk('saved_model')

print(f'NER labels: {ner.labels}')

# https://reposhub.com/python/deep-learning/explosion-spacy-transformers.html

Losses {'transformer': 120.0305118560791, 'tagger': 0.0, 'parser': 0.0, 'ner': 69.34346181201406}
Losses {'transformer': 133.47799158096313, 'tagger': 0.0, 'parser': 0.0, 'ner': 71.83022008263723}
Losses {'transformer': 144.30408477783203, 'tagger': 0.0, 'parser': 0.0, 'ner': 71.06096784855245}
Losses {'transformer': 106.79446744918823, 'tagger': 0.0, 'parser': 0.0, 'ner': 69.94046284099957}
Losses {'transformer': 123.32746028900146, 'tagger': 0.0, 'parser': 0.0, 'ner': 71.21936684130856}
Losses {'transformer': 273.9976296424866, 'tagger': 0.0, 'parser': 0.0, 'ner': 71.89628417004245}
Losses {'transformer': 150.05490255355835, 'tagger': 0.0, 'parser': 0.0, 'ner': 71.0601452096148}
Losses {'transformer': 120.09411811828613, 'tagger': 0.0, 'parser': 0.0, 'ner': 68.45370265407661}
Losses {'transformer': 108.89908123016357, 'tagger': 0.0, 'parser': 0.0, 'ner': 69.90267857886906}
Losses {'transformer': 120.18778419494629, 'tagger': 0.0, 'parser': 0.0, 'ner': 68.32772309139605}
Losses {'tran

In [14]:
# nlp = spacy.load('saved_model')

for s in sentences:
  doc = nlp(s)
  displacy.render(doc,jupyter=True,style="ent")
